今のところ、データ収集の部分のみが完了しています。

In [1]:
# libraryをimportする
import os
import re
import zipfile
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [12]:
# 江戸川乱歩の作者ページを設定する
edokawa_url = "https://www.aozora.gr.jp/index_pages/person1779.html"


# 青空文庫のページをUTF-8のencodingで取得する
response = requests.get(edokawa_url)
response.encoding = 'utf-8'

# BeautifulSoupを用いで、HTMLを解析する
soup = BeautifulSoup(response.text, 'html.parser')

# 作品リストを取得する
books_list = soup.select('ol li a')  

# 情報を抽出する
books = []
for book in books_list:
    relative_link = book['href'] 

    # 「person」が含まるlinkを無視する
    if "person" in relative_link:
        continue

    full_link = f"https://www.aozora.gr.jp{relative_link[2:]}"  # full linkに転換する
    book_name = book.text.strip()  # 作品名を収集する
    book_id = relative_link.split('/')[-1].replace('card', '').replace('.html', '')  # 作品IDを収集する

    # 作品の情報を基づいてdictionaryを作る
    books.append({book_id: [book_name, full_link]})

# dictionaryをprintする
for book in books:
    print(book)


{'57105': ['赤いカブトムシ', 'https://www.aozora.gr.jp/cards/001779/card57105.html']}
{'57181': ['赤い部屋', 'https://www.aozora.gr.jp/cards/001779/card57181.html']}
{'57240': ['悪魔の紋章', 'https://www.aozora.gr.jp/cards/001779/card57240.html']}
{'57515': ['悪霊', 'https://www.aozora.gr.jp/cards/001779/card57515.html']}
{'58699': ['悪霊物語', 'https://www.aozora.gr.jp/cards/001779/card58699.html']}
{'59399': ['「悪霊物語」自作解説', 'https://www.aozora.gr.jp/cards/001779/card59399.html']}
{'58333': ['暗黒星', 'https://www.aozora.gr.jp/cards/001779/card58333.html']}
{'57531': ['偉大なる夢', 'https://www.aozora.gr.jp/cards/001779/card57531.html']}
{'57182': ['一枚の切符', 'https://www.aozora.gr.jp/cards/001779/card57182.html']}
{'58053': ['一寸法師', 'https://www.aozora.gr.jp/cards/001779/card58053.html']}
{'57503': ['陰獣', 'https://www.aozora.gr.jp/cards/001779/card57503.html']}
{'56674': ['宇宙怪人', 'https://www.aozora.gr.jp/cards/001779/card56674.html']}
{'58590': ['江川蘭子', 'https://www.aozora.gr.jp/cards/001779/card58590.html']}
{'572

In [ ]:
def clean_and_extract_publication_year(text):
    # 出版した時間を抽出する
    cleaned_text = re.sub(r"（[^）]*）", "", text)
    pattern = r"(\d{4})"
    match = re.search(pattern, cleaned_text)
    if match:
        return match.group(1)
    return None

def extract_download_links(soup, base_url):
    # Webからすべての .zip download linkを抽出する
    download_links = []
    download_table = soup.find('table', class_='download')
    if download_table:
        download_rows = download_table.find_all('tr', bgcolor='white')
        for row in download_rows:
            link_tag = row.find('a', href=True)
            if link_tag and link_tag['href'].endswith('.zip'):
                full_url = urljoin(base_url, link_tag['href'])
                download_links.append(full_url)
    return download_links

def download_file(url, save_folder):
    # URLに従ってfileをdownloadし、指定されたfolderに保存する
    response = requests.get(url)
    response.raise_for_status()
    filename = url.split('/')[-1]
    save_path = os.path.join(save_folder, filename)
    os.makedirs(save_folder, exist_ok=True)
    with open(save_path, 'wb') as file:
        file.write(response.content)
    return save_path

def extract_zip(zip_path, extract_folder):
    # .zip fileを目的のfolderに解凍する
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
        return zip_ref.namelist() 

def rename_and_move_files(extracted_files, year, save_folder):
    # 解凍したfileの名前を変更して移動する
    renamed_files = []
    for file in extracted_files:
        old_path = os.path.join(save_folder, file)
        new_filename = f"{year}_{file}"
        new_path = os.path.join(save_folder, new_filename)
        os.rename(old_path, new_path)
        renamed_files.append(new_path)
    return renamed_files

def process_download_and_extract(url, save_folder, year):
    # .zipのfileをdownloadし、解凍して名前を変更する
    zip_path = download_file(url, save_folder)
    extracted_files = extract_zip(zip_path, save_folder)
    renamed_files = rename_and_move_files(extracted_files, year, save_folder)
    os.remove(zip_path)
    return renamed_files

# def process_books(book_dict_list, save_folder):
#     # すべてのprocessを統合する
#     for book_dict in book_dict_list:
#         for book_id, book_info in book_dict.items():
#             book_name, book_url = book_info  # 小説の情報を解析する
            
#             # ウェブページのHTMLを取得して解析する
#             response = requests.get(book_url)
#             response.encoding = 'utf-8'
#             soup = BeautifulSoup(response.text, 'html.parser')
            
#             # 初出年份を抽出する
#             try:
#                 initial_publication = soup.find('td', class_='header', string='初出：').find_next('td').get_text(strip=True)
#                 publication_year = clean_and_extract_publication_year(initial_publication)
#             except AttributeError:
#                 publication_year = "未知"
            
#             # download linkを抽出する
#             download_links = extract_download_links(soup, os.path.dirname(book_url) + '/')
            
#             # fileをdownloadし、解凍して名前を変更する
#             for link in download_links:
#                 renamed_files = process_download_and_extract(link, save_folder, publication_year)
#                 book_info.extend([publication_year] + renamed_files)

def process_books(book_dict_list, save_folder):
    # すべてのprocessを統合する
    for book_dict in book_dict_list:
        for book_id, book_info in book_dict.items():
            book_name, book_url = book_info  # 小説の情報を解析する
            
            # ウェブページのHTMLを取得して解析する
            response = requests.get(book_url)
            response.encoding = 'utf-8'
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 初出年份を抽出する
            try:
                initial_publication = soup.find('td', class_='header', string='初出：').find_next('td').get_text(strip=True)
                publication_year = clean_and_extract_publication_year(initial_publication)
            except AttributeError:
                publication_year = "未知"
            
            # download linkを抽出する
            download_links = extract_download_links(soup, os.path.dirname(book_url) + '/')
            
            # fileをdownloadし、解凍して名前を変更する
            for link in download_links:
                renamed_files = process_download_and_extract(link, save_folder, publication_year)
                book_info.extend([publication_year] + renamed_files)
    
    # 过滤掉 .png 文件路径
    for book in book_dict_list:
        for book_id, book_info in book.items():
            # 删除所有 .png 文件路径
            book_info = [info for info in book_info if not info.endswith('.png')]

save_folder = '/Users/yandongxin/Downloads/edokawa_text_data'

process_books(books, save_folder)

# 结果を表示する
for book in books:
    print(book)

57105 ['赤いカブトムシ', 'https://www.aozora.gr.jp/cards/001779/card57105.html', '1958', '/Users/yandongxin/Downloads/edokawa_text_data/1958_akai_kabutomushi.txt']
57181 ['赤い部屋', 'https://www.aozora.gr.jp/cards/001779/card57181.html', '1925', '/Users/yandongxin/Downloads/edokawa_text_data/1925_akai_heya.txt']
57240 ['悪魔の紋章', 'https://www.aozora.gr.jp/cards/001779/card57240.html', '1937', '/Users/yandongxin/Downloads/edokawa_text_data/1937_akumano_monsho.txt']
57515 ['悪霊', 'https://www.aozora.gr.jp/cards/001779/card57515.html', '1933', '/Users/yandongxin/Downloads/edokawa_text_data/1933_akuryo.txt']
58699 ['悪霊物語', 'https://www.aozora.gr.jp/cards/001779/card58699.html', '1954', '/Users/yandongxin/Downloads/edokawa_text_data/1954_akuryo_monogatari.txt']
59399 ['「悪霊物語」自作解説', 'https://www.aozora.gr.jp/cards/001779/card59399.html', '1954', '/Users/yandongxin/Downloads/edokawa_text_data/1954_akuryo_monogatari_jisaku_kaisetsu.txt']
58333 ['暗黒星', 'https://www.aozora.gr.jp/cards/001779/card58333.html',

In [14]:
def save_books_to_csv(book_dict_list, output_file):
    # list を CSV 形式に変換して保存する
    data = []
    
    for book_dict in book_dict_list:
        for book_id, book_info in book_dict.items():
            book_name = book_info[0]
            book_url = book_info[1]
            publication_year = book_info[2] if len(book_info) > 2 else "未知"
            extracted_files = book_info[3:] if len(book_info) > 3 else []
            extracted_files_str = "; ".join(extracted_files)
            
            data.append([book_id, book_name, book_url, publication_year, extracted_files_str])
    
    df = pd.DataFrame(data, columns=["Book ID", "Book Name", "Download Link", "Publication Year", "File Save Path"])
    df["text"] = ""
    df.to_csv(output_file, index=False, encoding="shift_jis")

csv_file_path = "/Users/yandongxin/Downloads/edokawa_text_data/books.csv"

save_books_to_csv(books, csv_file_path)

print(f"CSV fileが保存されている場所: {csv_file_path}")


CSV fileが保存されている場所: /Users/yandongxin/Downloads/edokawa_text_data/books.csv


In [15]:
def process_text(file_path):
    # 指定されたtext fileを処理し、不要な情報を削除する
    if pd.isna(file_path) or not os.path.exists(file_path):
        return None  # file が存在しない場合は None を返す

    # Shift_JIS でファイルを読み込む
    try:
        with open(file_path, "r", encoding="shift_jis", errors="ignore") as f:
            text = f.readlines()
    except Exception as e:
        print(f"ファイル読み込みエラー: {file_path} - {e}")
        return None

    # list形式のtextを文字列に変換
    text = [line.strip() for line in text]

    # 《》内の内容を削除
    text = [re.sub(r"《[^》]*》", "", line) for line in text]

    # ［＃］ 内の内容（前の数字も含む）を削除
    text = [re.sub(r"\d*［＃[^］]*］", "", line) for line in text]

    # ヘッダーの不要部分を削除（最後の `-------------------------------------------------------` の後の部分を残す）
    if "-------------------------------------------------------" in text:
        start_idx = len(text) - 1 - text[::-1].index("-------------------------------------------------------")
        text = text[start_idx + 1 :]

    # フッターの「底本：」以降を削除
    end_idx = next((i for i, line in enumerate(text) if line.startswith("底本：")), None)
    if end_idx is not None:
        text = text[:end_idx]

    # 空行を削除
    text = [line for line in text if line]

    # 処理後のテキストを改行で結合
    return "\n".join(text)


# CSV ファイルの読み込み（Shift_JIS）
csv_file = "/Users/yandongxin/Downloads/edokawa_text_data/books.csv"
df = pd.read_csv(csv_file, encoding="shift_jis")

# CSV に "file save path" 列があるか確認
if "File Save Path" not in df.columns:
    raise ValueError("CSV ファイルに 'File Save Path' 列がありません")

# 各ファイルを処理し、結果を "text" 列に格納
df["text"] = df["File Save Path"].apply(process_text)

# 処理後のデータを CSV に Shift_JIS で保存（行番号なし）
df.to_csv(csv_file, index=False, encoding="shift_jis")

print("テキスト処理が完了し、結果を Shift_JIS で CSV に保存しました。")

テキスト処理が完了し、結果を Shift_JIS で CSV に保存しました。


In [18]:
def check_text_column(csv_file):
    df = pd.read_csv(csv_file, encoding="shift_jis") 
    if "text" not in df.columns:
        print("csv fileの中に「text」列がない。")
        return

    empty_rows = df["text"].isna() | df["text"].str.strip().eq("")

    if empty_rows.any():
        print("「text」列は空であるか、スペースのみが含まれている行:")
        print(df[empty_rows])
    else:
        print("すべての行の「text」列の中に内容がある。")

csv_file = "/Users/yandongxin/Downloads/edokawa_text_data/books.csv"
check_text_column(csv_file)


「text」列は空であるか、スペースのみが含まれている行:
     Book ID Book Name                                      Download Link  \
3      57515        悪霊  https://www.aozora.gr.jp/cards/001779/card5751...   
13     57241      黄金仮面  https://www.aozora.gr.jp/cards/001779/card5724...   
20     56690  おれは二十面相だ  https://www.aozora.gr.jp/cards/001779/card5669...   
21     56673    怪奇四十面相  https://www.aozora.gr.jp/cards/001779/card5667...   
35     57185       黒手組  https://www.aozora.gr.jp/cards/001779/card5718...   
47     56646      心理試験  https://www.aozora.gr.jp/cards/001779/card5664...   
53     57106       大金塊  https://www.aozora.gr.jp/cards/001779/card5710...   
55     57502  探偵小説の「謎」  https://www.aozora.gr.jp/cards/001779/card5750...   
57     57533    智恵の一太郎  https://www.aozora.gr.jp/cards/001779/card5753...   
65     56687     天空の魔人  https://www.aozora.gr.jp/cards/001779/card5668...   
66     56689       電人Ｍ  https://www.aozora.gr.jp/cards/001779/card5668...   
67     57109    塔上の奇術師  https://www.aozora.gr.